# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [2]:
# Initial imports
import os
import pandas as pd
from newsapi import NewsApiClient
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/laurenkrohn/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Read your api key environment variable
load_dotenv()
api_key = os.getenv("news_api")

In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [18]:
# Fetch the Bitcoin news articles
btc_news = newsapi.get_everything(q="bitcoin or btc", language="en")
btc_news

{'status': 'ok',
 'totalResults': 1793,
 'articles': [{'source': {'id': None, 'name': 'Bitcoin.com'},
   'author': None,
   'title': 'Bitcoin, Ethereum Technical Analysis: BTC Slips to 10-Month Low, Below $33000 – Market Updates Bitcoin News - Bitcoin News',
   'description': '<ol><li>Bitcoin, Ethereum Technical Analysis: BTC Slips to 10-Month Low, Below $33000 – Market Updates Bitcoin News\xa0\xa0Bitcoin News\r\n</li><li>Bitcoin drops 50% from peak, near lowest levels in last one year\xa0\xa0Economic Times\r\n</li><li>Crypto Crash: Why is cryptoc…',
   'url': 'https://news.bitcoin.com/bitcoin-ethereum-technical-analysis-btc-slips-to-10-month-low-below-33000/',
   'urlToImage': 'https://static.news.bitcoin.com/wp-content/uploads/2022/05/shutterstock_1978798856-1.jpg',
   'publishedAt': '2022-05-09T13:45:30Z',
   'content': 'BTC fell to its lowest level since last July, as prices dropped below $33,000 to start the week. The selloff in cryptocurrency started last week, following the Fed’

In [16]:
# Fetch the Ethereum news articles
# NOTE: there were 0 news headlines about Ethereum today.
eth_news = newsapi.get_everything(q="ethereum", language="en")
eth_news

{'status': 'ok',
 'totalResults': 4661,
 'articles': [{'source': {'id': 'the-verge', 'name': 'The Verge'},
   'author': 'Emma Roth',
   'title': 'US blames North Korean hacker group for $625 million Axie Infinity theft',
   'description': 'The US Department of Treasury has attributed the $625 Axie Infinity theft to Lazarus, a North Korean hacking group with ties to the country’s intelligence agency.',
   'url': 'https://www.theverge.com/2022/4/14/23025739/north-korean-hacker-lazarus-axie-infinity-cryptocurrency-hack-theft-us-blames',
   'urlToImage': 'https://cdn.vox-cdn.com/thumbor/n8TGCqcud7UHPIbtgxLi10Owa5I=/0x52:1018x585/fit-in/1200x630/cdn.vox-cdn.com/uploads/chorus_asset/file/23374959/Screen_Shot_2022_03_30_at_10.10.00_AM.png',
   'publishedAt': '2022-04-14T21:43:57Z',
   'content': 'The US Department of Treasury says Lazarus is behind the attack\r\nThe US Treasury Department blames North Korean hacking group Lazarus for stealing $625 million in cryptocurrency from the Ronin netw

In [19]:
# Create the Bitcoin sentiment scores DataFrame
btc_sentiments = []

for article in btc_news["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        btc_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
btc_df = pd.DataFrame(btc_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
btc_df = btc_df[cols]

btc_df.head()

,date,text,compound,positive,negative,neutral
0,2022-05-09,"BTC fell to its lowest level since last July, ...",0.1027,0.076,0.066,0.859
1,2022-05-09,Did you hear? You may be able to allocate some...,0.1280,0.045,0.000,0.955
2,2022-04-19,<ul><li>Elon Musk will be able to focus on Tes...,0.4404,0.079,0.000,0.921
3,2022-05-03,Good morning. The combo of hawkish Fed policy ...,0.2732,0.170,0.118,0.713
4,2022-04-19,In a world where centralized finance is strugg...,0.2382,0.100,0.075,0.826


In [20]:
# Create the Ethereum sentiment scores DataFrame
eth_sentiments = []

for article in eth_news["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        eth_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
eth_df = pd.DataFrame(eth_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
eth_df = eth_df[cols]

eth_df.head()

,date,text,compound,positive,negative,neutral
0,2022-04-14,The US Department of Treasury says Lazarus is ...,-0.7845,0.088,0.231,0.681
1,2022-04-13,"18 with 15 posters participating, including st...",0.1027,0.084,0.071,0.844
2,2022-04-15,"Opera launched its dedicated ""Crypto Browser"" ...",0.7351,0.188,0.000,0.812
3,2022-05-02,A multi-billion dollar cryptocurrency company ...,-0.2263,0.046,0.075,0.879
4,2022-05-04,When Bored Ape Yacht Club creators Yuga Labs a...,-0.2732,0.000,0.055,0.945


In [22]:
# Describe the Bitcoin Sentiment
btc_df.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.228425,0.077000,0.027750,0.895350
std,0.271164,0.061003,0.040995,0.079955
min,-0.325200,0.000000,0.000000,0.713000
25%,0.077025,0.043750,0.000000,0.851000
50%,0.232250,0.076500,0.000000,0.907500
75%,0.422225,0.105500,0.045250,0.955000
max,0.834100,0.240000,0.125000,1.000000


In [23]:
# Describe the Ethereum Sentiment
eth_df.describe()

,compound,positive,negative,neutral
count,20.00000,20.00000,20.000000,20.000000
mean,-0.12974,0.04120,0.070250,0.888550
std,0.40896,0.05516,0.071528,0.080061
min,-0.78450,0.00000,0.000000,0.681000
25%,-0.31745,0.00000,0.000000,0.840250
50%,-0.24975,0.00000,0.063000,0.921000
75%,0.10270,0.07650,0.077750,0.937000
max,0.73510,0.18800,0.233000,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin had the highest average positive score, at 0.077, compared to the average positive score of 0.0412 for Ethereum.

Q: Which coin had the highest compound score?

A: Bitcoin had the highest (max) compound score, at 0.83 to Ethereum's 0.735.

Q. Which coin had the highest positive score?

A: Bitcoin had the highest (max) positive score, at 0.24, compared to the max positive score of 0.188 reached by Ethereum.

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [25]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [29]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
more_sw = []
sw = sw + more_sw
sw

TypeError: unsupported operand type(s) for +: 'set' and 'list'

In [12]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text

   
    # Create a tokenized list of the words
    
    
    # Lemmatize words into root words

   
    # Convert the words to lowercase
    
    
    # Remove the stop words
    
    
    return tokens

In [13]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

In [14]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [15]:
from collections import Counter
from nltk import ngrams

In [16]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [17]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---